<h1>
Week 11 - SQLAlchemy
<br>
<br>
<div>
<p>Surfs Up!</p>
</div>
</h1>


In [282]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [283]:
# sqlalchemy Dependencies
# ----------------------------------
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date

In [284]:
measurementsPath = os.path.join("resources", "hawaii_measurements.csv")
stationsPath = os.path.join("resources", "hawaii_stations.csv")

measurement_df = pd.read_csv(measurementsPath)
station_df = pd.read_csv(stationsPath)

In [285]:
clean_measurement_df = measurement_df.dropna()

In [286]:
clean_measurement_df.shape

(18103, 4)

In [287]:
clean_measurement_df.iloc[2].date
datetime.strptime(clean_measurement_df.iloc[2].date, '%Y-%m-%d')

datetime.datetime(2010, 1, 3, 0, 0)

In [288]:
clean_measurement_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [289]:
station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [290]:
station_df.name[1], station_df.iloc[0].station

('KANEOHE 838.1, HI US', 'USC00519397')

In [291]:
clean_measurement_df.to_csv("Output/clean_hawaii_measurements.csv", index=False, header=True)

In [292]:
station_df.to_csv("Output/clean_hawaii_stations.csv", index=False, header=True)

In [293]:
# Create Station and Measurement Class
# ----------------------------------
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float) 
    

In [294]:
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

In [295]:
Base.metadata.create_all(engine)

In [296]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [297]:
session.commit()

In [298]:
session.rollback()

In [299]:
for i in range(0, len(station_df)):
#     print(station_df.iloc[i].station)
    s = Station(station=station_df.iloc[i].station, name=station_df.name[i], latitude=station_df.iloc[i].latitude, 
                longitude = station_df.iloc[i].longitude, elevation=station_df.iloc[i].elevation )
    session.add(s)
    session.commit()
    

In [300]:
for i in range(0, len(clean_measurement_df)):
    m = Measurement(station=clean_measurement_df.iloc[i].station, 
                    date=datetime.strptime(clean_measurement_df.iloc[i].date, '%Y-%m-%d'),
                   prcp = clean_measurement_df.iloc[i].prcp, tobs=clean_measurement_df.iloc[i].tobs)
    session.add(m)
    session.commit()